In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
sys.path.insert(0, os.path.dirname(os.path.abspath("")))

In [3]:
import rlcard
from rlcard.agents import RandomAgent, DQNAgent

In [4]:
import tute.rlcard
from tute.rlcard.agent import TuteHumanAgent

In [5]:
import logging
tute.logger.setLevel(logging.INFO)

In [6]:
env = rlcard.make('tute')

In [7]:
print("Number of actions:", env.num_actions)
print("Number of players:", env.num_players)
print("Shape of state:", env.state_shape)
print("Shape of action:", env.action_shape)

Number of actions: 40
Number of players: 2
Shape of state: [[41, 9], [41, 9]]
Shape of action: [None, None]


### Human vs Random

In [43]:
random_agent = RandomAgent(num_actions=env.num_actions)
human_agent = TuteHumanAgent(player=1, tute=env.game)
env.set_agents([random_agent, human_agent])

In [44]:
trajectories, player_wins = env.run(is_training=False)

Player 1 swapped el cuatro de bastos for el dos de bastos
Player 1 played el seis de copas


================= Trump =================
el dos de bastos


================ Face up ================
el seis de copas


=============== Your Hand ===============
1: el tres de bastos
2: el seis de oros
3: el tres de copas
4: el siete de copas
5: el siete de espadas
6: el dos de copas
7: la sota de espadas
8: el tres de oros
1  2  3  4  5  6  7  8 ? 1

Player 2 played el tres de bastos
Player 2 won trick


================= Trump =================
el dos de bastos


=============== Your Hand ===============
1: el seis de oros
2: el tres de copas
3: el siete de copas
4: el siete de espadas
5: el dos de copas
6: la sota de espadas
7: el tres de oros
8: el as de bastos
1  2  3  4  5  6  7  8 ? 1

Player 2 played el seis de oros
Player 1 played el cuatro de bastos
Player 2 won trick


================= Trump =================
el dos de bastos


=============== Your Hand ===============
1: el tres 

In [45]:
player_wins

array([ 46, 104])

### Train Deep Q Network

In [46]:
agent = DQNAgent(
    num_actions=env.num_actions,
    state_shape=env.state_shape[0],
    mlp_layers=[64,64],
)

In [47]:
env.set_agents([agent, agent])

In [48]:
from rlcard.utils import (
    tournament,
    reorganize,
    Logger,
    plot_curve,
)

In [ ]:
with Logger(".") as logger:
    for episode in range(1000):

        # Generate data from the environment
        trajectories, payoffs = env.run(is_training=True)

        # Reorganaize the data to be state, action, reward, next_state, done
        trajectories = reorganize(trajectories, payoffs)

        # Feed transitions into agent memory, and train the agent
        for ts in trajectories[0]:
            agent.feed(ts)

        # Evaluate the performance.
        if episode % 50 == 0:
            logger.log_performance(
                env.timestep,
                tournament(
                    env,
                    100,
                )[0]
            )

In [ ]:
csv_path, fig_path = logger.csv_path, logger.fig_path
plot_curve(csv_path, fig_path, "DQN")

In [39]:
human_agent = TuteHumanAgent(player=1, tute=env.game)
env.set_agents([agent, human_agent])

In [ ]:
trajectories, player_wins = env.run(is_training=False)